# Dominating queens

As an example for the minimum dominating set problem, we find how many queens are needed and how to place them in order to keep all squares of an $n\times n$ chessboard in check.

In [ ]:
import networkx as nx

from itertools import product

# GraphILP API: import networkx graphs and use minimum dominating set
from graphilp.imports import networkx as nximp
from graphilp.covering import min_dom_set

In [ ]:
%matplotlib inline

In [ ]:
from matplotlib import pyplot as plt

# access to colour maps
from matplotlib import cm

# use patches to draw squares of the chessboard
import matplotlib.patches as patches

## Create the chessboard graph
We will create a graph with one vertex per square of the chessboard. A pair of squares will be connected by an edge if a queen in one of the squares checks the other one (this is symmetric).

In [ ]:
# choose the size of the chessboard
n = 8

In [ ]:
G = nx.Graph()

In [ ]:
# n by n squares
G.add_nodes_from(product(range(n), range(n)))

In [ ]:
# add the edges

for node in G.nodes():
    
    # check all squares in the same row
    edges = [(node, (node[0], i)) for i in range(n) if i != node[0]]
    
    # check all squares in the same column
    edges += [(node, (i, node[1])) for i in range(n) if i != node[1]]
    
    # check all squares in the same diagonal
    edges += [(node, (node[0]+i, node[1]+i)) for i in range(-max(node), max(n-1-node[0], n-1-node[1]))
              if (node[0]+i, node[1]+i) in G.nodes()]
    
    # check all squares in the same anti-diagonal
    edges += [(node, (node[0]+i, node[1]-i)) for i in range(-max(node), max(node))
              if ((node[0]+i, node[1]-i) in G.nodes()) and ((node[0]+i, node[1]-i) != node)]
    
    G.add_edges_from(edges)

## Set up and solve optimisation problem

Set up the minimum dominating set problem using GraphILP API:

In [ ]:
optG = nximp.read(G)

In [ ]:
m = min_dom_set.createModel(optG)

Find a minimum dominating set in the chessboard graph:

In [ ]:
m.optimize()

In [ ]:
queens = min_dom_set.extractSolution(optG, m)

## Plot the result

Find out which square is checked by which queens

In [ ]:
covered = {}

for q in queens:
    for e in G.edges(q):
        if e[1] in covered:
            covered[e[1]].append(q)
        else:
            covered[e[1]] = [q]

Assign colours to the queens

In [ ]:
queen_colors = {}
c = 0
for q in queens:
    queen_colors[q] = cm.get_cmap('Set1')(c)
    c += 1

Plot the chessboard with the queens and show which square is checked by which queen

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = plt.subplot(111, aspect='equal')

for node in G.nodes():
    sq = patches.Rectangle(node, 1, 1, fill=True, color = 'k' if sum(node)%2==0 else 'w')
    ax.add_patch(sq)
    
for c, clist in covered.items():
    if c not in queens:
        sq = patches.Rectangle((3+n+c[0]+0.2, c[1]+0.2), 0.6, 0.6, fill=True, color = queen_colors[clist[0]])
        ax.add_patch(sq)

for q in queens:
    plt.plot(q[0]+0.5, q[1]+0.5, 'o', color=queen_colors[q])
    plt.plot(3+n+q[0]+0.5, q[1]+0.5, 'o', color=queen_colors[q])
    
ax.set_axis_off()
ax.relim()
ax.autoscale_view()    